In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, Dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(777)
torch.cuda.manual_seed_all(777)
#====================================================================
# calling data (3x32x32)
trans = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset = torchvision.datasets.CIFAR10(root='/home/gus/PyTorch',
                                        train=True,
                                        download=True,
                                        transform=trans)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=32,
                                          shuffle=True,
                                          num_workers=2)
testset = torchvision.datasets.CIFAR10(root='/home/gus/PyTorch', 
                                       train=False,
                                       download=True, 
                                       transform=trans)
testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=32,
                                         shuffle=False,
                                         num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
#====================================================================
# def imshow(img):
#     img = img / 2 + 0.5     # unnormalize
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()

# # 학습용 이미지를 무작위로 가져오기
# dataiter = iter(trainloader)
# images, labels = dataiter.next()

# # 이미지 보여주기
# imshow(torchvision.utils.make_grid(images))
# # 정답(label) 출력
# print(' '.join('%5s' % classes[labels[j]] for j in range(32)))
#====================================================================
# declaring hyperparameters
lr = 0.01
#====================================================================
# constructiong LeNet-5 model
class Lenet(nn.Module):
    def __init__(self):
        super(Lenet,self).__init__()
        # input shape = (?, 3, 32, 32)
        # Conv1 -> (?, 16, 32, 32)
        # Pool1 -> (?, 16, 16, 16)
        # Conv2 -> (?, 32, 16, 16)
        # Pool2 -> (?, 32, 8, 8)
        # Conv2 -> (?, 1, 8, 8)
        self.layer1 = nn.Sequential(
            torch.nn.Conv2d(3,16,5,stride=1,padding=2),
            torch.nn.Tanh(),
            torch.nn.AvgPool2d(kernel_size=2,stride=2),
            torch.nn.Conv2d(16,32,5,stride=1,padding=2),
            torch.nn.Tanh(),
            torch.nn.AvgPool2d(kernel_size=2,stride=2),
            torch.nn.Conv2d(32,1,5,stride=1,padding=2),
            torch.nn.Tanh()
        )
        self.layer2 = nn.Sequential(
            torch.nn.Linear(1*8*8,84,bias=True),
            torch.nn.Tanh(),
            torch.nn.Linear(84,10,bias=True)
        )
        
    def forward(self,x):
        out = self.layer1(x)
        out = out.view(out.size(0),-1)
        out = self.layer2(out)
        return out
#====================================================================
model = Lenet().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = lr)
    
for epoch in range(20):
    avg_cost=0
    for X,Y in trainloader:
        X = X.to(device)
        Y = Y.to(device)
            
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
            
        avg_cost += cost/10
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

Files already downloaded and verified
Files already downloaded and verified
[Epoch:    1] cost = 313.117249
[Epoch:    2] cost = 304.788879
[Epoch:    3] cost = 299.784576
[Epoch:    4] cost = 298.065277
[Epoch:    5] cost = 298.475922
[Epoch:    6] cost = 294.628601
[Epoch:    7] cost = 292.906616
[Epoch:    8] cost = 291.465027
[Epoch:    9] cost = 292.038696
[Epoch:   10] cost = 291.551453
[Epoch:   11] cost = 292.034485
[Epoch:   12] cost = 293.070374
[Epoch:   13] cost = 290.413269
[Epoch:   14] cost = 293.285614
[Epoch:   15] cost = 297.315033
[Epoch:   16] cost = 292.558411
[Epoch:   17] cost = 289.369751
[Epoch:   18] cost = 296.248383
[Epoch:   19] cost = 291.927673
[Epoch:   20] cost = 295.426636


In [5]:
with torch.no_grad():
    a = 0
    for X,Y in testloader:
        X = X.to(device)
        Y = Y.to(device)
        prediction = model(X)
        correct_prediction = torch.argmax(prediction, 1) == Y
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())
        a += 1
        print(a)
        if a == 10:
            break


Accuracy: 0.375
1
Accuracy: 0.375
2
Accuracy: 0.28125
3
Accuracy: 0.40625
4
Accuracy: 0.15625
5
Accuracy: 0.25
6
Accuracy: 0.25
7
Accuracy: 0.3125
8
Accuracy: 0.21875
9
Accuracy: 0.53125
10
